# Fase 3: EDA

In [67]:
import pandas as pd
import numpy as np

import sys
import os

# Agregar el directorio 'src' al path
sys.path.append(os.path.abspath('../src'))

# Importar la función suma del módulo auxiliares
import soporte_limpieza as sl

In [68]:
df = pd.read_csv("../datos/tabla_limpia.csv", parse_dates=["fecha recaudacion"])
df.shape

(987202, 16)

### 1- Distribución de Ingresos por Categoría Económica:

In [69]:
df["categoria economica"].value_counts()

categoria economica
Receitas Correntes                           925088
Receitas de Capital                           28311
Sem informação                                18532
Receitas Correntes - intra-orçamentárias      15183
Receitas de Capital - intra-orçamentárias        88
Name: count, dtype: int64

Analizar las categorías de ingresos más significativas y su participación en los ingresos totales.

Las más significativas son en cuanto a valor recaudado pues es lo que realmente nos interesa ya que es el dinerito que realmente entra a la caja.

In [70]:
df.groupby("categoria economica").sum(numeric_only=True)["valor recaudado"].sort_values(ascending=False)

categoria economica
Receitas Correntes                           1.202508e+13
Receitas de Capital                          1.200415e+13
Sem informação                               3.271665e+11
Receitas Correntes - intra-orçamentárias     2.805787e+11
Receitas de Capital - intra-orçamentárias    2.141127e+10
Name: valor recaudado, dtype: float64

In [71]:
l_aux = list(df.groupby("categoria economica").sum(numeric_only=True)["valor recaudado"].sort_values(ascending=False))
total_categorias =0
for valor in l_aux:
    total_categorias+=valor
total_categorias

24658378453467.75

In [72]:
np.round(df.groupby("categoria economica").sum(numeric_only=True)["valor recaudado"].sort_values(ascending=False)/24658378453467.75*100,2)

categoria economica
Receitas Correntes                           48.77
Receitas de Capital                          48.68
Sem informação                                1.33
Receitas Correntes - intra-orçamentárias      1.14
Receitas de Capital - intra-orçamentárias     0.09
Name: valor recaudado, dtype: float64

Hay dos categorías que son las que más recaudan claramente que son Receitas Correntes y de Capital con casi el 50% cada una de la recaudación.

Ahora queremos ver la diferencia promedio entre ingresos previstos y realizados por cada categoría.

In [146]:
df_comparacion = df.groupby("categoria economica").sum(numeric_only=True)[["valor previsto","valor recaudado"]]
df_comparacion["diferencia"] = df_comparacion['valor recaudado']-df_comparacion['valor previsto']

In [147]:
df_comparacion_rounded = df_comparacion / 1000000000000 # pasamos a billones
df_comparacion_rounded

,valor previsto,valor recaudado,diferencia
categoria economica,,,
Receitas Correntes,13.144754,12.025076,-1.119678
Receitas Correntes - intra-orçamentárias,0.269428,0.280579,0.011151
Receitas de Capital,13.499860,12.004146,-1.495714
Receitas de Capital - intra-orçamentárias,0.010808,0.021411,0.010603
Sem informação,1.016380,0.327166,-0.689213


In [148]:
df_comparacion["diferencia"].mean()

np.float64(-656570396059.7698)

In [149]:
df_comparacion["diferencia"].median()

np.float64(-689213379070.6699)

Por lo general, la diferencia entre recaudado y previsto suele ser negativa ya que hay dos valores negativos muy grandes, otro negativo mediano y dos positivos bajos. Por lo que de media se recauda medio billón menos de lo previsto.

### 2- Análisis Temporal:

In [76]:
df.columns

Index(['Unnamed: 0', 'id organizacion superior', 'organizacion superior',
       'id organizacion', 'organizacion', 'id unidad gestora',
       'unidad gestora', 'categoria economica', 'origen ingreso',
       'tipo ingreso', 'valor previsto', 'valor registrado', 'valor recaudado',
       'porcentaje recaudado', 'fecha recaudacion', 'anio recaudacion'],
      dtype='object')

In [77]:
df_anios = df.groupby("anio recaudacion").sum(numeric_only=True)[["valor previsto", "valor registrado", "valor recaudado"]]
df_anios = df_anios / 1000000000000 # pasamos a billones

In [81]:
df_anios["recaudado-previsto"]= df_anios["valor recaudado"]-df_anios["valor previsto"]
df_anios["recaudado-registrado"]= df_anios["valor recaudado"]-df_anios["valor registrado"]

In [82]:
df_anios

,valor previsto,valor registrado,valor recaudado,recaudado-previsto,recaudado-registrado
anio recaudacion,,,,,
2013.0,2.128724,1.137824,1.663430,-0.465293,0.525606
2014.0,2.333104,1.258080,2.211514,-0.121590,0.953434
2015.0,2.871164,0.306336,2.634219,-0.236945,2.327883
2016.0,2.910191,0.331444,2.787181,-0.123010,2.455737
2017.0,3.400684,0.453527,2.476073,-0.924610,2.022546
2018.0,3.476635,0.468685,2.865977,-0.610658,2.397292
2019.0,3.249178,0.339548,2.880424,-0.368754,2.540876
2020.0,3.528192,0.623802,3.460501,-0.067691,2.836699
2021.0,4.043359,0.356910,3.679059,-0.364300,3.322149


In [83]:
df_anios["recaudado-previsto"].mean()

np.float64(-0.3647613311443164)

In [84]:
df_anios["recaudado-registrado"].mean()

np.float64(2.1535803234023847)

En cuanto al valor recaudado podemos observar como ha ido aumentando a lo largo de los años, a excepción de 2017 donde hubo un pequeña caida respecto al año anterior. Además se pegó un gran salto entre 2019 y 2020 donde se recaudo medio billón más con respecto al año anterior. También se puede onservar como a partir de 2015 el valor que se refleja como registrado no es de apenas medio billon cuando se está recaudando en algunos casos hasta 5,6 o y veces más. Lo que nos plantea, ¿el dinero recaudado pero no resgitrado que pasa con él? ¿Se va a actualizar el registro o ese dinero desaparece? Porque es muchísimo dinero. Además, todo los años el previsto es mucho mayor que el recaudado, ¿por qué calculan siempre mal el previsto?

In [98]:
df_anios_meses = df.groupby([df["fecha recaudacion"].dt.year, df["fecha recaudacion"].dt.month]).sum(numeric_only=True)[["valor previsto", "valor registrado", "valor recaudado"]]

In [103]:
df_anios_meses_rounded = np.round(df_anios_meses/100000000000, 8)

In [102]:
pd.set_option("display.max_rows", None)
df_anios_meses_rounded

valor previsto  valor registrado  \
fecha recaudacion fecha recaudacion                                     
2013.0            12.0                    20.787542         11.378244   
2014.0            12.0                    23.130657         11.698561   
2015.0            12.0                    28.628689          3.062643   
2016.0            1.0                     29.096459          0.036278   
                  2.0                      0.000000          0.124155   
                  3.0                      0.000000          0.171638   
                  4.0                      0.000000          0.032135   
                  5.0                      0.000000         -0.098063   
                  6.0                      0.000000         -0.083749   
                  7.0                      0.000000         -0.224849   
                  8.0                      0.000498          0.000000   
                  9.0                      0.000000          0.170242   
                  10.0                     0.000000          0.000000   
                  11.0                     0.000000          0.084348   
                  12.0                     0.000000          0.092137   
2017.0            1.0                     33.890403         -0.059663   
                  2.0                      0.000000          0.000000   
                  3.0                     -0.000495          0.023418   
                  4.0                      0.000000          0.070992   
                  5.0                      0.000000          0.205158   
                  6.0                      0.000000          0.000000   
                  7.0                      0.000531         -0.002201   
                  8.0                      0.000000          0.677607   
                  9.0                      0.000000          0.614890   
                  10.0                     0.000000         -0.545842   
                  11.0                     0.000000          0.002302   
                  12.0                     0.000000          0.289284   
2018.0            1.0                     34.489641         -0.035418   
                  2.0                      0.000000          0.037194   
                  3.0                      0.000000          0.048723   
                  4.0                      0.000000         -0.061045   
                  5.0                      0.000000         -0.001316   
                  6.0                      0.000000         -0.129895   
                  7.0                      0.000000          0.075652   
                  8.0                      0.000436          0.000700   
                  9.0                      0.000000         -0.003286   
                  10.0                     0.000000          0.000491   
                  11.0                     0.000000          0.183299   
                  12.0                     0.000000          0.111181   
2019.0            1.0                     32.464351          0.094564   
                  2.0                      0.000000         -0.590260   
                  3.0                      0.000000         -0.143965   
                  4.0                      0.000000         -0.084447   
                  5.0                      0.000000         -0.010967   
                  6.0                      0.000000         -0.126207   
                  7.0                      0.000000         -0.229805   
                  8.0                      0.000000          0.049797   
                  9.0                      0.000000         -0.146601   
                  10.0                     0.000000          0.030177   
                  11.0                     0.000476         -0.173727   
                  12.0                     0.000000          0.120363   
2020.0            1.0                     34.421501          0.000214   
                  2.0                      0.000000         -0.606524   
                  3.0 

En cuanto a los 3 primero años que son 2013,2014 y 2015 solo hay registros del mes de diciembre. 

Para el resto de los años hay registro de todos los meses, se observa que el valor previsto por lo general solo tiene un valor coherente en el mes de enero, a excepción de 2021 que es en el mes de abril. Por lo general el resto de los meses suele estar a 0 excepción de uno donde hay un valor previsto bastante más bajo en comparación con el de enero. Esto lo que me parece indicar es que establecen un valor a principio de año de lo que se va a obtener en todo el año y puede que hagan algún pequeño ajuste a mediados de año.

En cuanto al valor registrado no hay datos que se puedan abstarear de forma genérica. cada mes por lo general se registra un valor a excepción de algunos que son 0, este valor puede ser positivo o negativo indicando que igual han surgido gatos y ese mes se han quedado en negativo. El valor registrado se podría estudiar más en profundidad pero ya de forma individual para cada año pero no parece seguir ningún patrón.

El valor recaudado para cada mes siempre es positivo y superior al valor registrado.

In [116]:
df_meses = df.groupby( df["fecha recaudacion"].dt.month).sum(numeric_only=True)[["valor previsto", "valor registrado", "valor recaudado"]]

In [119]:
df_meses_rounded = np.round(df_meses/1000000000, 8)

In [123]:
df_meses_rounded.sort_values(by = "valor recaudado", ascending=False)

,valor previsto,valor registrado,valor recaudado
fecha recaudacion,,,
12.0,7254.688791,2686.431534,7817.338526
9.0,0.000000,60.934716,1917.900669
10.0,0.000000,-52.084147,1896.495277
8.0,0.093477,86.654679,1734.176606
1.0,16436.235421,3.595165,1724.540968
4.0,3963.254357,16.291642,1492.304783
7.0,0.101407,-59.721963,1467.127359
3.0,-0.049471,8.281396,1339.403669
11.0,0.047581,13.304972,1294.132894


Si agrupamos todos los valores de todos los años solo por meses se ve claramente como en diciembre es el mes que se recauda más con gran diferencia, esto se debe a que en 2013,2014 y 2015 solo se recauda en diciembre. el siguiente mes que más recauda es septiembre y octubre ya que puede que en estos mese es cuando ellos hagan su versión de declaración de la renta.

### 3-Identificación de Discrepancias por categorías.

Recordamos la tabla del primer paso:

In [150]:
df_comparacion = df.groupby("categoria economica").sum(numeric_only=True)[["valor previsto","valor recaudado"]]
df_comparacion["diferencia"] = df_comparacion['valor recaudado']-df_comparacion['valor previsto']

In [152]:
df_comparacion_rounded = df_comparacion / 1000000000000 # pasamos a billones
df_comparacion_rounded.sort_values(by="diferencia")

,valor previsto,valor recaudado,diferencia
categoria economica,,,
Receitas de Capital,13.499860,12.004146,-1.495714
Receitas Correntes,13.144754,12.025076,-1.119678
Sem informação,1.016380,0.327166,-0.689213
Receitas de Capital - intra-orçamentárias,0.010808,0.021411,0.010603
Receitas Correntes - intra-orçamentárias,0.269428,0.280579,0.011151


Las categorías con mayor diferencia entre lo recaudado y lo previsto son Receitas de Capital y Receitas Corrientes, estas son las dos mayores categorías de ingresos que obtienen casi el 50% cada una. Es complicado prever cuanto dinero van a obtener esta categorías recaudadoras sobre todo siendo las que gestionan la mitad de la recaudación del país. Estando presentes ante una subejecución por parte de ambas categorías. Sin embargo, Receitas de Capital - intra-orçamentárias y Receitas Correntes - intra-orçamentárias si que se aproximan mucho mejor al valor recaudado.

Tendría que estudiarse que esta ocurriendo con las dos primeras y por que se estiman recaudar más de un billón de lo que realmente se recauda.

Vamos a filtrar por años a ver si vemos algo más claro:

In [156]:
df_anio_categoria = df.groupby(["anio recaudacion", "categoria economica"]).sum(numeric_only=True)[["valor previsto","valor recaudado"]]
df_anio_categoria["diferencia"] = df_anio_categoria['valor recaudado']-df_anio_categoria['valor previsto']
df_anio_categoria = df_anio_categoria / 1000000000000 # pasamos a billones

# Solo quiero las dos preocupantes
categorias_a_seleccionar = ['Receitas Correntes', 'Receitas de Capital']
df_anio_categoria_filtrado = df_anio_categoria[df_anio_categoria.index.get_level_values('categoria economica').isin(categorias_a_seleccionar)]

In [157]:
df_anio_categoria_filtrado

valor previsto  valor recaudado  \
anio recaudacion categoria economica                                    
2013.0           Receitas Correntes         1.223951         0.961517   
                 Receitas de Capital        0.855343         0.635259   
2014.0           Receitas Correntes         1.322853         1.220860   
                 Receitas de Capital        0.979554         0.947027   
2015.0           Receitas Correntes         1.467130         1.244528   
                 Receitas de Capital        1.356511         1.332119   
2016.0           Receitas Correntes         1.448261         1.318764   
                 Receitas de Capital        1.421812         1.403917   
2017.0           Receitas Correntes         1.430729         1.344009   
                 Receitas de Capital        1.002567         1.062380   
2018.0           Receitas Correntes         1.491126         1.462242   
                 Receitas de Capital        1.949908         1.343625   
2019.0           Receitas Correntes         1.613195         1.622519   
                 Receitas de Capital        1.597268         1.212610   
2020.0           Receitas Correntes         1.661065         1.281706   
                 Receitas de Capital        1.829874         2.039198   
2021.0           Receitas Correntes         1.486445         1.568931   
                 Receitas de Capital        2.507023         2.028011   

                                      diferencia  
anio recaudacion categoria economica              
2013.0           Receitas Correntes    -0.262434  
                 Receitas de Capital   -0.220083  
2014.0           Receitas Correntes    -0.101993  
                 Receitas de Capital   -0.032527  
2015.0           Receitas Correntes    -0.222601  
                 Receitas de Capital   -0.024392  
2016.0           Receitas Correntes    -0.129497  
                 Receitas de Capital   -0.017895  
2017.0           Receitas Correntes    -0.086720  
                 Receitas de Capital    0.059813  
2018.0           Receitas Correntes    -0.028884  
                 Receitas de Capital   -0.606283  
2019.0           Receitas Correntes     0.009324  
                 Receitas de Capital   -0.384657  
2020.0           Receitas Correntes    -0.379360  
                 Receitas de Capital    0.209324  
2021.0           Receitas Correntes     0.082486  
                 Receitas de Capital   -0.479012

No hay nada del todo claro, igual podemos decir que en 2019 y 2021 las receitas Correntes empezaron a acercarse un poco mejor al valor recaudado. Para receitas Capital en 2020 se estimo un poco de menos y llego 2021 y estimaron medio billón de más.

## Conclusiones

- En cuanto a categorías tenemos dos claras categorias que recaudan cada una casi el 50% de valor recaudado que son `Receitas Correntes` y `Receitas de Capital`. Estudiando la diferencia promedio entre los ingresos previstos y recaudados observamos que para estas do categorías siempre se prevee más de lo que se recauda y de hecho la media de las diferenias es negativa, es decir, de media se ha hecho una previsión erronea por categoría de entorno a medio billón.

- En cuanto al análisis temporal se observa que la recaudación ha ido en aumento cada año, habiendo un aumento significativo en 2020. Sin embargo, ocurre de nuevo que para todos los años el valor previsto es siempre mayor que el recaudado, recaudando de media un tercio me billón menos de lo esperado cada año. Por otro lado la diferencia entre recaudado y previsto es siempre positiva, y creciendo cada año, un claro indicativo de que no se están registrando todas las cuentas y media obteniendo cada año 2 billones mas de lo registrado, haciendonos la pregunta de, si este dinero no está registrado, dónde está. 

    En cuanto a los meses, 2013, 2014 y 2015 solo había valores del mes de diciembre por lo que no sorprendeque el mes que más aparece que se recauda si agrupamos por meses es diciembre, seguido de septiembre y octubre. El valor previsto parece ser establecido siempre en enero estando casi todos los demás meses a 0 con algún pequeño ajuste otro mes del año. En cuanto al valor registrado siempre es menor cada mes que el valor recaudado, siendo algunos meses incluso negativo.

- Hay dos clara categorías que presentan una subejecucción y son `Receitas Correntes` y `Receitas de Capital`, recaudando a lo largo de los años un billón menos de lo calculado. Por lo que un problema que se debe revisar como se está realizando esta estimación y por qué si todos los años fallán no aprenden de sus errores y mejoran algo.